In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils as U
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import random

### Define the Neural Network

We want to be able to set the weight of the network manually

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, in_dim=2, out_dim=2):
        super(NeuralNet, self).__init__()
        
        self.layer1=nn.Linear(in_features=in_dim, out_features=4)
        self.layer2=nn.Linear(in_features=4, out_features=out_dim)

        self.weights_initialization()
    
    def weights_initialization(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)  # This is the default in PyTorch
                nn.init.constant_(module.bias, 0)

    def forward(self, x):
        out = self.layer1(x)
        out = F.relu(out)
        out = self.layer2(out)
        return out
    
    def get_flat_params(self):
        return U.parameters_to_vector(self.parameters())
    
    def set_flat_params(self, flat_params):
        U.vector_to_parameters(flat_params, self.parameters())

### Load Data

In [3]:
X, y = make_moons(n_samples=1000, noise=0.1, random_state=42)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

### Evolutionary Algorithm

The parameters are the genes.
The flat vector of parameters is the chromosome.
The population is the different random combinations of parameters.

In [ ]:
# Initialize model and get flat parameters
model = NeuralNet()
params_vector = model.get_flat_params()
print(params_vector)

# Configuration
POPULATION_SIZE = 100
CHROMOSOME_LENGTH = len(params_vector)

tensor([ 5.4157e-01, -9.6938e-01,  6.8159e-01, -3.3370e-01,  3.3292e-01,
         6.7682e-01,  9.4453e-01, -1.3159e-01,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  7.5796e-02, -2.5452e-01,  1.1230e-01,
         9.0249e-01, -1.3721e-04, -2.8852e-01,  7.4950e-01,  8.8189e-01,
         0.0000e+00,  0.0000e+00], grad_fn=<CatBackward0>)


In [44]:
def generate_population(size, chromosome_length):
    """ Generate random population """
    return [torch.randn(chromosome_length) for _ in range(size)]

def fitness(chromosome, model):
    """ Algorithm's fitness function """
    model.set_flat_params(chromosome)   # Update model's parameters with the chromosome
    with torch.no_grad():
        outputs = model(X_test)
        pred = outputs.argmax(dim=1)
        accuracy = (pred == y_test).float().mean().item()
    return accuracy

def selection(population, fitness_scores):
    """ Roulette Wheel Selection """
    total_fitness = sum(fitness_scores)

    if total_fitness == 0:  # If it's 0 we get an uniform distribution
        selection_probs = [1 / len(fitness_scores)] * len(fitness_scores)
    else:
        selection_probs = [f / total_fitness for f in fitness_scores]
    
    parent1 = population[random.choices(range(len(population)), selection_probs)[0]]
    parent2 = population[random.choices(range(len(population)), selection_probs)[0]]
    return parent1, parent2

def crossover(parent1, parent2):
    """Two-Point Crossover"""
    p1, p2 = sorted(random.sample(range(1, len(parent1)), 2))
    offspring1 = torch.cat([parent1[:p1], parent2[p1:p2], parent1[p2:]])
    offspring2 = torch.cat([parent2[:p1], parent1[p1:p2], parent2[p2:]])
    return offspring1, offspring2

# def mutate_v1(chromosome, mutation_rate=0.01):
#     """ Mutation Function """
#     noise = torch.randn_like(chromosome) * mutation_rate # Generate random tensor with same length as chromosome
#     return chromosome + noise

def mutate(chromosome, mutation_rate=0.05, mutation_strength=0.1):
    """
        Mutate a small subset of the genes of the chromosome.

        Args:
            chromosome (tensor): Chromosome to mutate.
            mutation_rate (float): Probability of mutating each gene.
            mutation_strength (float): How much noise is aggregated to mutated genes.
    """
    mutated = chromosome.clone()
    for i in range(len(mutated)):
        if random.random() < mutation_rate:
            mutated[i] += torch.randn(1).item() * mutation_strength
    return mutated



In [47]:
population = generate_population(POPULATION_SIZE, CHROMOSOME_LENGTH)
fitness_scores = [fitness(individual, model) for individual in population]
parent1, parent2 = selection(population, fitness_scores)
offspring1, offspring2 = crossover(parent1, parent2)
mutated_offspring1 = mutate(offspring1)

print(f"Population Sample: {population[0]}\n")
print(f"Fitness Score Sample: {fitness_scores[0]}\n")
print(f"Selection Result: {parent1, parent2}\n")
print(f"Crossover Result: {offspring1, offspring2}\n")
print(f"Mutated Offspring 1: {mutated_offspring1}")


Population Sample: tensor([-1.4109, -0.8019, -0.5627, -0.3699, -0.1248,  0.7096,  1.3216, -1.6099,
         0.7494,  0.1980, -0.3541,  0.9934,  0.4551, -0.7793,  0.2918,  1.0571,
         0.8362, -1.4426, -0.1471, -1.3692, -2.7578, -0.6193])

Fitness Score Sample: 0.2150000035762787

Selection Result: (tensor([ 0.6360, -2.6240,  1.4052, -0.5129,  0.4750,  0.0873, -1.1254,  0.9676,
         0.5300, -0.3612,  0.1463, -1.1278, -1.1146, -2.5379, -0.2693, -0.9066,
         3.0045,  1.2147, -0.7319, -0.8432, -0.3637,  1.4939]), tensor([-2.3640, -0.3271, -1.4925,  1.3096,  2.0729,  0.9872,  0.7629,  1.2031,
        -0.7920, -0.3794, -0.1867,  1.3370,  0.6505, -1.7242,  0.5505, -0.9173,
        -1.3901,  0.4659, -1.5331,  0.5608,  0.9323,  0.8407]))

Crossover Result: (tensor([ 0.6360, -2.6240,  1.4052, -0.5129,  2.0729,  0.9872,  0.7629,  1.2031,
        -0.7920, -0.3794, -0.1867,  1.3370,  0.6505, -1.7242,  0.5505, -0.9173,
        -1.3901,  0.4659, -1.5331,  0.5608, -0.3637,  1.4939]), tens